In [1]:
%pip install torch
%pip install torchvision
%pip install torchsummary

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

In [3]:
#choose device
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu')

In [4]:
#define transform
train_transform=transforms.Compose(
    [transforms.Pad(4),
     transforms.RandomCrop(32),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485,0.456,0.406],
                          std=[0.229,0.224,0.225])])
valid_transform=transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485,0.456,0.406],
                          std=[0.229,0.224,0.225])])

In [5]:
# download cifar-10 dataset
train_dataset=torchvision.datasets.CIFAR10(root='./data/',
                                           train=True,
                                           transform=train_transform,
                                           download=True)
test_dataset=torchvision.datasets.CIFAR10(root='./data/',
                                           train=False,
                                           transform=valid_transform,
                                           download=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=out_channels,stride=stride,kernel_size=3,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)
        self.conv2=nn.Conv2d(in_channels=out_channels,out_channels=out_channels,stride=1,kernel_size=3,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.downsample=nn.Sequential()
        if (stride!=1) or(in_channels!=out_channels):
            self.downsample=nn.Sequential(
                nn.Conv2d(in_channels=in_channels,out_channels=out_channels,stride=stride,kernel_size=1,bias=False),
                nn.BatchNorm2d(out_channels))
    def forward(self,x):
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        out=self.conv2(out)
        out=self.bn2(out)
        out+=self.downsample(x)
        out=self.relu(out)
        return out
        

In [7]:
# 4-layer ResNet
class ResNet(nn.Module):
    def __init__(self,block,layers,num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels=64                                                  # 64 can be modified
        self.conv=nn.Conv2d(in_channels=3,out_channels=64,stride=1,kernel_size=3,padding=1,bias=False)
        self.bn=nn.BatchNorm2d(64)
        self.relu=nn.ReLU(inplace=True)
        self.layer1=self.make_layer(block, 64,layers[0], stride=1)        # 64 can be modified
        self.layer2=self.make_layer(block, 128, layers[1], stride=2)      # 128 can be modified
        self.layer3=self.make_layer(block, 256, layers[2], stride=2)      # 256 can be modified
        #self.layer4=self.make_layer(block, 512, layers[3], stride=2)      # 512 can be modified
        self.avg_pool=nn.AvgPool2d(8)                                        # 4 can be modified
        self.fc=nn.Linear(256,10) 
        
    def make_layer(self,block,out_channels,blocks, stride):
        layers=[]
        layers.append(block(self.in_channels,out_channels,stride))
        self.in_channels=out_channels
        for i in range(1,blocks):
            layers.append(block(out_channels,out_channels,stride=1))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out=self.conv(x)
        out=self.bn(out)
        out=self.relu(out)
        out=self.layer1(out)
        out=self.layer2(out)
        out=self.layer3(out)
        #out=self.layer4(out)
        out=self.avg_pool(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [8]:
#train
def train(model):
    model.train()
    train_loss=0
    train_acc=0
    for img,label in train_loader:
        img=img.to(device)
        label=label.to(device)
        #forward
        output=model(img)
        loss=criterion(output,label)
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #calculate loss and acc
        train_loss+=loss.item()
        _, predicted=output.max(1)
        train_acc+=(predicted==label).sum().item()/len(predicted)
    return train_loss/len(train_loader),train_acc/len(train_loader)

In [9]:
def test(model):
    model.eval()
    with torch.no_grad():
        valid_acc=0
        for img,label in test_loader:
            img=img.to(device)
            label=label.to(device)
            output=model(img)
            _, predicted=output.max(1)
            valid_acc+=(predicted==label).sum().item()/len(output)
    return valid_acc/len(test_loader)

In [10]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=128,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=128,shuffle=False)
model=ResNet(ResidualBlock,[2,2,2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9, weight_decay=1e-3)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=50)
for i in range(50):
    train_loss, train_acc=train(model)
    test_acc=test(model)
    print("Epoch %d. Train Loss %.4f. Train Acc %.4f. Test Acc %.4f."%(i,train_loss,train_acc,test_acc))
    #scheduler.step()

Epoch 0. Train Loss 1.4228. Train Acc 0.4765. Test Acc 0.5623.
Epoch 1. Train Loss 0.9493. Train Acc 0.6640. Test Acc 0.6750.
Epoch 2. Train Loss 0.7523. Train Acc 0.7337. Test Acc 0.7291.
Epoch 3. Train Loss 0.6285. Train Acc 0.7807. Test Acc 0.7509.
Epoch 4. Train Loss 0.5507. Train Acc 0.8093. Test Acc 0.7903.
Epoch 5. Train Loss 0.4959. Train Acc 0.8308. Test Acc 0.7472.
Epoch 6. Train Loss 0.4502. Train Acc 0.8444. Test Acc 0.7815.
Epoch 7. Train Loss 0.4169. Train Acc 0.8556. Test Acc 0.8183.
Epoch 8. Train Loss 0.3871. Train Acc 0.8671. Test Acc 0.7978.
Epoch 9. Train Loss 0.3635. Train Acc 0.8744. Test Acc 0.8447.
Epoch 10. Train Loss 0.3394. Train Acc 0.8846. Test Acc 0.7932.
Epoch 11. Train Loss 0.3215. Train Acc 0.8895. Test Acc 0.8437.
Epoch 12. Train Loss 0.3077. Train Acc 0.8945. Test Acc 0.8249.
Epoch 13. Train Loss 0.2904. Train Acc 0.8993. Test Acc 0.8454.
Epoch 14. Train Loss 0.2721. Train Acc 0.9075. Test Acc 0.8670.
Epoch 15. Train Loss 0.2620. Train Acc 0.9104. Tes